# Expanded Image Analysis Lab

## Instructions
Using one of these packages:
1. imager 
2. EBImage from bioc 
3. ripa 
4. adimpro


Options are:

[tasks here]

With one or more of these images  (or use your own):

[image choices from web and sample images folder]



perform an image processing task and then visualize.

In [3]:
library(jpeg)

imgX = readPNG(file.path(images, "5_Expanded_Image_Analysis_Lab", "poor_image.jpg"))
x = matrix(imgX, nrow(imgX) * ncol(imgX), 1)
y = coef(lm.mod.1)[1] + coef(lm.mod.1)[2] * x
y[y < 0] = 0
y[y > 1] = 1
img = matrix(y, nrow(imgX), ncol(imgX))
img.dt=data.table(melt(img))
names.dt<-names(img.dt)
setnames(img.dt,names.dt[1],"X1")
setnames(img.dt,names.dt[2],"X2")
# img.dt[,id:= "1"]
write.table(img.dt[,c("id","value"),with=FALSE], file = "submission.csv", sep = ",",col.names = (f == filenames[1]),row.names = FALSE, quote = FALSE, append = (f != filenames[1]))
    

ERROR: Error in eval(expr, envir, enclos): could not find function "readPNG"


ERROR: Error in matrix(imgX, nrow(imgX) * ncol(imgX), 1): object 'imgX' not found


ERROR: Error in coef(lm.mod.1): object 'lm.mod.1' not found


ERROR: Error in y[y < 0] = 0: object 'y' not found


ERROR: Error in y[y > 1] = 1: object 'y' not found


...

ERROR: Error in eval(expr, envir, enclos): could not find function "data.table"


ERROR: Error in eval(expr, envir, enclos): object 'img.dt' not found


ERROR: Error in eval(expr, envir, enclos): could not find function "setnames"


ERROR: Error in eval(expr, envir, enclos): could not find function "setnames"


ERROR: Error in is.data.frame(x): object 'img.dt' not found


## NNs

* On visualizing NNs: https://beckmw.wordpress.com/tag/nnet/
